# 作業
請使用不同的資料集，並使用 hyper-parameter search 的方式，看能不能找出最佳的超參數組合

In [32]:
from sklearn import datasets, metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV

In [33]:
# 匯入資料
digits = datasets.load_digits()

# 將資料區分特徵與目標
x = digits.data
y = digits.target

# 查看資料型態
print("Shape of x:", x.shape)
print("Shape of y:", y.shape)

Shape of x: (1797, 64)
Shape of y: (1797,)


In [36]:
# 切割資料成train與test
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state = 50)

# 建立模型
clf = RandomForestClassifier(n_estimators=10, random_state=3)

# 訓練模型
clf.fit(x_train, y_train)

# 查看訓練分數
print("Score of train", clf.score(x_train, y_train))

Score of train 0.9992044550517104


In [37]:
# 預測test資料
y_pred = clf.predict(x_test)

# 計算準確率
acc = metrics.accuracy_score(y_pred, y_test)
print("Accuracy of Base Model:",acc)

Accuracy of Base Model: 0.9296296296296296


In [40]:
## 進行超參數的調整
# 設定要訓練的超參數組合
n_estimators = [10, 20, 30, 50, 100]
max_depth = [1, 3, 5, 10, 20, 30]
param_grid = dict(n_estimators=n_estimators, max_depth=max_depth)

## 建立搜尋物件，放入模型及參數組合字典 (n_jobs=-1 會使用全部 cpu 平行運算)
grid_search = GridSearchCV(clf, param_grid, scoring="neg_mean_squared_error", n_jobs=-1, verbose=1, cv=3, iid=True)

# 開始搜尋最佳參數
grid_result = grid_search.fit(x_train, y_train)

# 預設會跑 3-fold cross-validadtion，總共 30 種參數組合，總共要 train 90 次模型

Fitting 3 folds for each of 30 candidates, totalling 90 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  52 tasks      | elapsed:    0.9s
[Parallel(n_jobs=-1)]: Done  90 out of  90 | elapsed:    1.6s finished


In [41]:
# 印出最佳結果與最佳參數
print("Best Accuracy: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best Accuracy: -0.404932 using {'max_depth': 10, 'n_estimators': 100}


In [42]:
# 最佳化結果是{'max_depth': 10, 'n_estimators': 50}
# 尚未釐清 Accuracy: -0.519491 意思, 待未來有時間去了解

In [47]:
# 使用最佳參數重新建立模型
clf_bestparam = RandomForestClassifier(max_depth=grid_result.best_params_['max_depth'],
                                       n_estimators=grid_result.best_params_['n_estimators'], random_state=3)

# 訓練模型
clf_bestparam.fit(x_train, y_train)

# 預測
y_pred = clf_bestparam.predict(x_test)

In [48]:
# 計算準確率
acc = metrics.accuracy_score(y_pred, y_test)
print("Accuracy of Best Model:",acc)

Accuracy of Best Model: 0.9592592592592593


準確率  
Accuracy of Base Model: 0.9296296296296296  
提升至  
Accuracy of Best Model: 0.9592592592592593  